In [1]:
import sys 

sys.path.append('../lib/nameh-khan')

## Import 

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

from nameh_khan.page_reader.utils.segmentation import SimpleSegmentation

## Config

In [3]:
NUMBER_OF_PAGES = 20

## Data Path

In [4]:
DATASET_DIR_TEMPLATE = 'raw-dataset/zar/{0:05d}'

RAW_IMAGE_PATH = '{}/page.png'.format(DATASET_DIR_TEMPLATE)
RAW_LABEL_PATH = '{}/label.csv'.format(DATASET_DIR_TEMPLATE)

## Data Preparation 

In [5]:
for i in range(1, NUMBER_OF_PAGES+1):
    image = cv2.imread(RAW_IMAGE_PATH.format(i), flags=cv2.IMREAD_GRAYSCALE)
    segmentation = SimpleSegmentation()
    box_locations = segmentation.transform(image)
    label_pdf = (
        pd.DataFrame(box_locations)
        .sort_values(by=['y1'], ascending=[True])
        .assign(lineChange=lambda x: (x.y1 > x.y2.shift(1)).astype(int))
        .assign(lineNumber=lambda x: x.lineChange.cumsum())
        .assign(word = '')
        .sort_values(by=['lineNumber', 'x2'], ascending=[True, False])
        .reset_index(drop=True)
        .loc[:, ['word', 'x1', 'y1', 'x2', 'y2']]
    )

    label_pdf.to_csv(RAW_LABEL_PATH.format(i), index=False)